# Stock Trades by Members of the US House of Representatives

This project uses public data about the stock trades made by members of the US House of Representatives. This data is collected and maintained by Timothy Carambat as part of the [House Stock Watcher](https://housestockwatcher.com/) project. The project describes itself as follows:

> With recent and ongoing investigations of incumbent congressional members being investigated for potentially violating the STOCK act. This website compiles this publicly available information in a format that is easier to digest then the original PDF source.
>
> Members of Congress must report periodic reports of their asset transactions. This website is purely for an informative purpose and aid in transparency.
>
> This site does not manipluate or censor any of the information from the original source. All data is transcribed by our community of contributors, which you can join for free by going to our transcription tool. Our moderation team takes great care in ensuring the accuracy of the information.
>
> This site is built and maintained by Timothy Carambat and supported with our contributors.

Some interesting questions to consider for this data set include:

- Is there a difference in stock trading behavior between political parties? For example:
    - does one party trade more often?
    - does one party make larger trades?
    - do the two parties invest in different stocks or sectors? For instance, do Democrats invest in Tesla more than Republicans?
- What congresspeople have made the most trades?
- What companies are most traded by congresspeople?
- Is there evidence of insider trading? For example, Boeing stock dropped sharply in February 2020. Were there a suspiciously-high number of sales of Boeing before the drop?
- When are stocks bought and sold? Is there a day of the week that is most common? Or a month of the year?

### Getting the Data

The full data set of stock trade disclosures is available as a CSV or as JSON at https://housestockwatcher.com/api.

This data set does not, however, contain the political affiliation of the congresspeople. If you wish to investigate a question that relies on having this information, you'll need to find another dataset that contains it and perform a merge. *Hint*: Kaggle is a useful source of data sets.


### Cleaning and EDA

- Clean the data.
    - Certain fields have "missing" data that isn't labeled as missing. For example, there are fields with the value "--." Do some exploration to find those values and convert them to null values.
    - You may also want to clean up the date columns to enable time-series exploration.
- Understand the data in ways relevant to your question using univariate and bivariate analysis of the data as well as aggregations.


### Assessment of Missingness

- Assess the missingness per the requirements in `project03.ipynb`

### Hypothesis Test / Permutation Test
Find a hypothesis test or permutation test to perform. You can use the questions at the top of the notebook for inspiration.

# Summary of Findings

### Introduction
TODO

### Cleaning and EDA
TODO

### Assessment of Missingness
TODO

### Hypothesis Test
TODO

# Code

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import missingno as msno
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

### Cleaning and EDA

In [2]:
fp = os.path.join('data/', 'all_transactions.csv')
all_transactions = pd.read_csv(fp)
all_transactions.isna().sum()

disclosure_year              0
disclosure_date              0
transaction_date             0
owner                     4529
ticker                       0
asset_description            4
type                         0
amount                       0
representative               0
district                     0
ptr_link                     0
cap_gains_over_200_usd       0
dtype: int64

So let us begin by finding all potential fake missingness and turning it into real missingness. It appears, after some rigorous analysis (looking at each series potential values), that the only fake missingness exists in the 'owner' and 'ticker' columns. We will now replace those values with NaNs. In summary:

Missingness in data:
- Owner column has fake missing ('--') and real missing
- Ticker column has fake missing ('--')
- Asset description column has real missing but only 4 so we can hand clean this

In [3]:
all_transactions.iloc[732] #Replace NaN in a_d with Ball Corporation
all_transactions.iloc[3262] #Replace NaN in a_d with CELO
all_transactions.iloc[10537] #Replace NaN in a_d with URGO
all_transactions.iloc[10538] #Replace NaN in a_d with URGO

disclosure_year                                                        2021
disclosure_date                                                  03/18/2021
transaction_date                                                 2021-02-18
owner                                                                   NaN
ticker                                                                 URGO
asset_description                                                       NaN
type                                                               purchase
amount                                                     $1,001 - $15,000
representative                                              Hon. Brian Mast
district                                                               FL18
ptr_link                  https://disclosures-clerk.house.gov/public_dis...
cap_gains_over_200_usd                                                False
Name: 10538, dtype: object

In [4]:
all_transactions.get('asset_description').loc[732] = 'Ball Corporation'
all_transactions.get('asset_description').loc[3262] = 'CELO'
all_transactions.get('asset_description').loc[10537] = 'URGO'
all_transactions.get('asset_description').loc[10538] = 'URGO'

C:\Users\vdand\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Our current strategy for tickers is to ignore the ticker and instead use the asset description wherever we can. The ticker is currently hypothesized to be NMAR after some brief exploration of some missing tickers: Missing dependent upon if the asset is listed on the NASDAQ. This can be further proven by understanding that the collection process populates the form according to NASDAQ tickers. 

In the following cell we sorted the transaction date values to find the smallest values from a string perspective, and discovered several anomalies. For these anomalies our intention is to hand clean the data and make sure that the date listed matches the date reported in the PDF sent in to Congress. 

In [5]:
all_transactions['transaction_date'].sort_values().head(10)

2069    0009-06-09
9381    0021-06-22
3496    0021-08-02
9382    0201-06-22
8662    2012-06-19
3489    2018-09-08
3488    2018-09-09
9627    2018-12-27
7875    2019-01-09
7535    2019-01-09
Name: transaction_date, dtype: object

In [6]:
all_transactions.iloc[8662] #Correct transaction date is: 06/19/2020
all_transactions.iloc[2069] #Correct transaction date is: 06/09/2021
all_transactions.iloc[9381] #Correct transaction date is: 06/22/2021
all_transactions.iloc[3496] #Correct transaction date is: 08/02/2021
all_transactions.iloc[9382] #Correct transaction date is: 06/22/2021

disclosure_year                                                        2021
disclosure_date                                                  07/16/2021
transaction_date                                                 0201-06-22
owner                                                                   NaN
ticker                                                                   KR
asset_description                                            Kroger Company
type                                                           sale_partial
amount                                                     $1,001 - $15,000
representative                                       Hon. James E Hon Banks
district                                                               IN03
ptr_link                  https://disclosures-clerk.house.gov/public_dis...
cap_gains_over_200_usd                                                False
Name: 9382, dtype: object

In [7]:
all_transactions.get('transaction_date').loc[8662] = '2020-06-19'
all_transactions.get('transaction_date').loc[2069] = '2021-06-09'
all_transactions.get('transaction_date').loc[9381] = '2021-06-22'
all_transactions.get('transaction_date').loc[3496] = '2021-08-02'
all_transactions.get('transaction_date').loc[9382] = '2021-06-22'

In [8]:
all_transactions['transaction_date'].sort_values().head(10)

3489    2018-09-08
3488    2018-09-09
9627    2018-12-27
7475    2019-01-09
7535    2019-01-09
7797    2019-01-09
7469    2019-01-09
7560    2019-01-09
7420    2019-01-09
7570    2019-01-09
Name: transaction_date, dtype: object

In [9]:
transactions_w_nan = all_transactions.replace('--', np.nan)

In [10]:
transactions_w_nan.isna().sum()

disclosure_year              0
disclosure_date              0
transaction_date             0
owner                     5844
ticker                    1023
asset_description            0
type                         0
amount                       0
representative               0
district                     0
ptr_link                     0
cap_gains_over_200_usd       0
dtype: int64

In [11]:
transactions_w_nan['transaction_date'] = pd.to_datetime(transactions_w_nan['transaction_date'])
transactions_w_nan['transaction_date'].dtypes

dtype('<M8[ns]')

In [12]:
transactions_w_nan['disclosure_date'] = pd.to_datetime(transactions_w_nan['disclosure_date'])
transactions_w_nan['disclosure_date'].dtypes

dtype('<M8[ns]')

In [13]:
transactions_w_nan[transactions_w_nan['owner'] != 'dependent']['ptr_link'].iloc[450]

'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2021/20019443.pdf'

In [27]:
owner_isna = set(transactions_w_nan[transactions_w_nan['owner'].isna()]['representative'].unique())
owner_isnotna = set(transactions_w_nan[~transactions_w_nan['owner'].isna()]['representative'].unique())
in_both = list(owner_isna.intersection(owner_isnotna))[0]

In [34]:
transactions_w_nan[transactions_w_nan['representative'] == in_both].groupby(['owner', 'disclosure_date'], dropna=False).count()

disclosure_year  transaction_date  ticker  \
owner disclosure_date                                              
joint 2020-01-06                    31                31      31   
      2020-02-10                    36                36      36   
      2020-03-16                    46                46      46   
      2020-04-11                    55                55      55   
      2020-05-13                    29                29      29   
      2020-06-10                    28                28      28   
      2020-07-09                    41                41      41   
      2020-08-10                    17                17      17   
      2020-09-20                    27                27      27   
      2020-10-13                    55                55      55   
      2020-11-05                    50                50      50   
      2020-12-09                    62                62      62   
      2021-01-07                    23                23      23   
      2021-02-09                    11                11      11   
      2021-03-10                    59                59      58   
      2021-04-21                    64                64      64   
      2021-07-15                    23                23      23   
      2021-08-09                    30                30      30   
      2021-09-20                    20                20      20   
      2021-10-13                    32                32      32   
self  2020-10-13                     1                 1       0   
NaN   2020-03-16                     3                 3       3   
      2020-04-11                     6                 6       6   
      2020-06-10                     6                 6       6   
      2020-10-13                     1                 1       1   

                       asset_description  type  amount  representative  \
owner disclosure_date                                                    
joint 2020-01-06                      31    31      31              31   
      2020-02-10                      36    36      36              36   
      2020-03-16                      46    46      46              46   
      2020-04-11                      55    55      55              55   
      2020-05-13                      29    29      29              29   
      2020-06-10                      28    28      28              28   
      2020-07-09                      41    41      41              41   
      2020-08-10                      17    17      17              17   
      2020-09-20                      27    27      27              27   
      2020-10-13                      55    55      55              55   
      2020-11-05                      50    50      50              50   
      2020-12-09                      62    62      62              62   
      2021-01-07                      23    23      23              23   
      2021-02-09                      11    11      11              11   
      2021-03-10                      59    59      59              59   
      2021-04-21                      64    64      64              64   
      2021-07-15                      23    23      23              23   
      2021-08-09                      30    30      30              30   
      2021-09-20                      20    20      20              20   
      2021-10-13                      32    32      32              32   
self  2020-10-13                       1     1       1               1   
NaN   2020-03-16                       3     3       3               3   
      2020-04-11                       6     6       6               6   
      2020-06-10                       6     6       6               6   
      2020-10-13                       1     1       1               1   

                       district  ptr_link  cap_gains_over_200_usd  
owner disclosure_date                                              
joint 2020-01-06             31        31          

In [38]:
transactions_w_nan[(transactions_w_nan['representative'] == in_both) & (transactions_w_nan['disclosure_date'] == '2020-03-16')]['ptr_link'].iloc[0]

'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2020/20016181.pdf'

### Assessment of Missingness

In [6]:
# TODO

### Hypothesis Test / Permutation Test

In [ ]:
# TODO